# Warp a disk

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
%matplotlib widget

## General Definitions of disk surface and warp parameters

Define surface:

$z = h_1 \, r^{1+hr\_index}$

In [ ]:
h1 = 0.05
hr_index = 0.25

Define the warp (inclination for each ring): logistic function from $i=0$ outside transitioning to `warp_max` at radius `r0` over a transition width of `dr`.

In [ ]:
warp_max = np.pi / 6
dr = 0.1
r0 = 1

Calculate the interfaces and cell centers of the disk surface

In [ ]:
nphi = 50 # number of points along azimuth
nr  = 20  # number of rings

# define cylindrical radius, azimuthal angle, and height above mid plane
r   = np.ones([nr, nphi]) * np.linspace(.2, 2, nr)[:, None]
phi = np.ones([nr, nphi]) * np.linspace(0, 2 * np.pi, nphi)
z   = h1 * r**(1 + hr_index)

# define centers as well
rc   = 0.5 * (r[1:, 1:] + r[:-1, 1:])
zc   = h1 * rc**(1 + hr_index)
phic = 0.5 * (phi[1:, 1:] + phi[1:, :-1])

# convert to cartesian (x, y = edges, xc, yc = centers)

x   = r * np.cos(phi)
y   = r * np.sin(phi)

xc   = rc * np.cos(phic)
yc   = rc * np.sin(phic)

In [ ]:
# calculate warp inclinations
warp   = warp_max / (1.0 + np.exp((r[:, 0] - r0) / dr))
warp_c = 0.5 * (warp[1:] + warp[:-1])

Making an array of the coordinates that is `(nr, nphi, 3)`

In [ ]:
points = np.moveaxis([x, y, z], 0, 2)
points_c = np.moveaxis([xc, yc, zc], 0, 2)

Calculate the velocities of all centers

In [ ]:
v = (rc**2 + zc**2)**-0.25
v = v[None, :, :] * np.array([-np.sin(phic), np.cos(phic), np.zeros_like(phic)])
v = np.moveaxis(v, 0, 2)

Define each radius' twist

In [ ]:
twist_0 = -np.pi * 3 / 4 # this is the general rotation of the entire disk around the original axis
twist = np.linspace(0, 20, nr) * np.pi / 180. + twist_0
twist_c = 0.5 * (twist[1:] + twist[:-1]) # twist of the cell centers

define the inclination of the observer as rotation around x

In [ ]:
inc_obs = 0 * np.pi / 4
PA_obs = 0

## Step-by-step process

Here we do things step by step in the notebook. Below is using the functions inside `helper`.

Define the rotation matrices

In [ ]:
import helper
Rx = helper.Rx
Ry = helper.Ry
Rz = helper.Rz

apply the ring inclination

In [ ]:
points1 = np.einsum('ijk,klj->kli', Ry(warp), points)
points1_c = np.einsum('ijk,klj->kli', Ry(warp_c), points_c)
v1 = np.einsum('ijk,klj->kli', Ry(warp_c), v)

apply the twist

In [ ]:
points2 = np.einsum('ijk,klj->kli', Rz(twist), points1)
points2_c = np.einsum('ijk,klj->kli', Rz(twist_c), points1_c)
v2 = np.einsum('ijk,klj->kli', Rz(twist_c), v1)

#### 3D Plot of original and new rings

In [ ]:
with plt.rc_context({'lines.linewidth': 1}):
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(projection='3d')

    #ax.plot(x, y, z)

    for i, _points in enumerate([points2]):

        for ir in range(nr):
            _x, _y, _z = _points[ir].T
            ax.plot(_x, _y, _z, c=f'C{i}')

        for iphi in range(nphi):
            _x, _y, _z = _points[:, iphi, :].T
            ax.plot(_x, _y, _z, c=f'C{i}')
            
    for iphi in range(nphi - 1):
        _x, _y, _z = points2_c[:, iphi, :].T
        ax.plot(_x, _y, _z, ls='none', marker='.', mfc='k', mec='none', markersize=1.5)
    
    # vectors
    scale = 0.05
    arr = np.array([points2_c, points2_c + scale * v2])
    for ir in range(nr - 1):
        for iphi in range(nphi - 1):
            _x, _y, _z =  arr[:, ir, iphi, :].T
            ax.plot(_x, _y, _z, 'r-')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_zlim([-2, 2])
    ax.axis('off')

### Observer Projection

Default observer is face on, so image plane normal would be (0,0,-1). And the base vectors are
$$
\vec e_x = \pmatrix{1\\0\\0},
\vec e_y = \pmatrix{0\\1\\0}, 
\vec e_z = \pmatrix{0\\0\\-1}\\
$$

In [ ]:
ex = (1, 0, 0)
ey = (0, 1, 0)
ez = (0, 0, -1)

base = np.array([ex, ey, ez])

rotate the base

In [ ]:
R_inc = Rx(inc_obs)
R_PA = Rz(PA_obs)

base2 = R_inc[:, :, 0].dot(R_PA[:, :, 0].dot(base))

project the points

In [ ]:
points3 = np.einsum('ijk,kl->ijl', points2, base2)
points3_c = np.einsum('ijk,kl->ijl', points2_c, base2)
v3 =  np.einsum('ijk,kl->ijl', v2, base2)

#### 2D plot of projected points

In [ ]:
with plt.rc_context({'lines.linewidth': .5}):
    f, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)

    for ir in range(nr):
        _x, _y, _z = points3[ir].T
        ax.plot(_x, _y, c='C0')

    for iphi in range(nphi):
        _x, _y, _z = points3[:, iphi, :].T
        ax.plot(_x, _y, c='C0')

    for iphi in range(nphi - 1):
        _x, _y, _z = points3_c[:, iphi, :].T
        ax.plot(_x, _y, ls='none', marker='.', mfc='k', mec='none', markersize=1.5)

    # plot the vectors: z is used as color
    scale = 0.05
    arr = np.array([points3_c, points3_c + scale * v3])
    col = Normalize()(v3[:, :, -1]) 
    cmap = plt.cm.RdBu_r
    
    for ir in range(nr - 1):
        for iphi in range(nphi - 1):
            _x, _y, _z =  arr[:, ir, iphi, :].T
            ax.plot(_x, _y, color=cmap(col[ir, iphi]), lw=2)

In [ ]:
plt.close('all')